In [ ]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromDir
from Eval.Clouds.Sphere import CreateBaseSpherePointCloud
from Eval.Error import CalculateRMSE, CalculateSTD
from Eval.Plotting import get_histogram_standards
from Utils.Viz import VizualiseBaseTargetPointclouds
from Utils.Format import MilimeterToMeter, MeterToMilimeter

In [ ]:
# Create Base Clouds for Comparison, Created Programatically for High Accuracy.

# Diameter of Base IRL Sphere: 200mm
baseSphere = CreateBaseSpherePointCloud(diameter = MilimeterToMeter(200), resolution=5000)
baseSphere.paint_uniform_color([1, 1, 1])

In [ ]:
# Current Technique and Data in Review

eval = 'Point_E' # Either Kinect, PolyCam or Point_E

evalDataDir = f'../data/{eval}/sphere.ply'

In [ ]:
# Get the Obtained Data Clouds for Comparison, on the Base Programatically Created Clouds.

collectedSphere = CreatePointcloudFromDir(evalDataDir)
collectedSphere.paint_uniform_color([1.0, 0.0, 0.0])

In [ ]:
# Perform Data Adjustment based on know problems with Each technique.

match eval:
    case 'Kinect':
        print("Performing Kinect Data Adjustment")
        collectedSphere = collectedSphere.uniform_down_sample(every_k_points=5)
    
    case 'PolyCam':
        print("Performing PolyCam Data Adjustment")
        collectedSphere = collectedSphere.uniform_down_sample(every_k_points=10)

    case 'Point_E':
        print("Performing Point_E Data Adjustment")

        print("Scaling Point_E Data to Match Meter Units")
        collectedSphere.scale(MilimeterToMeter(200), collectedSphere.get_center())


In [ ]:
# Perform ICP Registration to Obtain a Transformation Matrix
# This allows for more accurate comparison of the clouds, by making them similar positioning.


icpRegResults = o3d.pipelines.registration.registration_icp(collectedSphere, 
                                                            baseSphere, 
                                                            0.1, 
                                                            np.identity(4), 
                                                            o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
                                                            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))

print(icpRegResults)

# Apply the Transformation Matrix to the Obtained Data Clouds
collectedSphere.transform(icpRegResults.transformation)

In [ ]:
# Quick Visualization of both the Obtained Cloud and the Base Cloud.

VizualiseBaseTargetPointclouds(baseSphere, collectedSphere)

In [ ]:
# Calculate the RMSE between the two clouds.

rmse = CalculateRMSE(collectedSphere, baseSphere)

# Convert to mm
rmse *= 1000

print(f'RMSE (mm): {rmse}')

In [ ]:
# Calculate the STD between the two clouds.

stdData = CalculateSTD(collectedSphere, baseSphere)

print(f'STD (m): {stdData}')

# Convert to mm
stdData["std"] *= 1000
stdData["mean"] *= 1000

stdData["min"] *= 1000
stdData["max"] *= 1000
##

print(f'STD (mm): {stdData}')

In [ ]:
# Check Histogram Standards

max_density, max_dist = get_histogram_standards(collectedSphere, baseSphere)

print (f'Max Prob Density: {max_density}, Max Dist: {max_dist}')

### Save Data

In [ ]:
from Utils.Data import JSON

In [ ]:
rmsejsonDir = "../data/techniques-rmse-data.json"
stdjsonDir = "../data/techniques-std-data.json"

rmse_data = JSON(rmsejsonDir)
std_data = JSON(stdjsonDir)

In [ ]:
# Update Stored RMSE Data

rmse_data.data[eval]["Sphere"].update({
    "rmse": rmse,
    "inlier_rmse": MeterToMilimeter(icpRegResults.inlier_rmse)
})

rmse_data.print()
rmse_data.save()


In [ ]:
# Update Stored STD Data
std_data.data[eval]["Sphere"].update(stdData)

# Update Stored STD Standards
histStandards = std_data.data["STANDARDS"]

if (max_density > histStandards["MAX_PROB_DENSITY"]):
    histStandards["MAX_PROB_DENSITY"] = float(max_density)
    print(f"New Max Prob Density: {max_density}")

if (max_dist > histStandards["MAX_DISTANCE"]):
    histStandards["MAX_DISTANCE"] = float(max_dist)
    print(f"New Max Distance: {max_dist}")

std_data.data["STANDARDS"].update(histStandards)

# Set the Max Frequency
std_data.print()
std_data.save()